In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

@njit
def diffeq_accuracy(dy, t, y):
    dy[0] = np.sin(t) - y[1]  # dydt = sin(t) - x(t)
    dy[1] = np.cos(t) + y[0]  # dxdt = cos(t) + y(t)
    
@njit
def diffeq_accuracy_2(t, y, dy):
    dy[0] = np.sin(t) - y[1]  # dydt = sin(t) - x(t)
    dy[1] = np.cos(t) + y[0]  # dxdt = cos(t) + y(t)

@njit
def diffeq_accuracy_3(t, y):
    dy = np.empty(y.shape, dtype=np.float64)
    dy[0] = np.sin(t) - y[1]  # dydt = sin(t) - x(t)
    dy[1] = np.cos(t) + y[0]  # dxdt = cos(t) + y(t)
    return dy

@njit
def correct_answer(t, c1_, c2_):
    y = np.empty((2, t.size), dtype=np.float64)
    y[0] = -c1_ * np.sin(t) + c2_ * np.cos(t) - (np.cos(t) / 2)  # -c1 * sin(t) + c2 * cos(t) - cos(t) / 2
    # At t=0; y = c2 - 1/2
    y[1] = c2_ * np.sin(t) + c1_ * np.cos(t) + (np.sin(t) / 2)   # c2 * sin(t) + c1 * cos(t) + sin(t) / 2
    # At t=0; x = c1
    return y

# Initial Conditions
# y=0 --> c2 = + 1/2
c2 = 0.5
# x=1 --> c1 = + 1
c1 = 1.0
y0 = np.asarray((0., 1.), dtype=np.float64)
time_span_ = (0., 10.)

In [2]:
from CyRK import pysolve_ivp, cyrk_ode
from scipy.integrate import solve_ivp

In [5]:
rtols = [1.0e-5, 1.0e-7, 1.0e-9, 1.0e-11]

integration_method = 'RK45'
rk_method = 1
t_span = (0., 10.)
t_eval = np.linspace(0.0, 10.0, 100)

for rtol in rtols:
    atol = rtol/100
    rtol = 1.0e-8
    atol = 1.0e-9
    
    print("\n RTOL = ", rtol)
    
    scipy_sol   = solve_ivp(diffeq_accuracy_3, t_span, y0, rtol=rtol, atol=atol, method=integration_method)
    scipy_teval = solve_ivp(diffeq_accuracy_3, t_span, y0, rtol=rtol, atol=atol, method=integration_method, t_eval=t_eval)
    
    cyrk_sol   = cyrk_ode(diffeq_accuracy_2, t_span, y0, rtol=rtol, atol=atol, rk_method=rk_method, raise_warnings=False)
    cyrk_teval = cyrk_ode(diffeq_accuracy_2, t_span, y0, rtol=rtol, atol=atol, rk_method=rk_method, t_eval=t_eval, raise_warnings=False)
    
    pysolve_sol   = pysolve_ivp(diffeq_accuracy, t_span, y0, rtol=rtol, atol=atol, method=integration_method, dense_output=True, pass_dy_as_arg=True)
    pysolve_teval = pysolve_ivp(diffeq_accuracy, t_span, y0, rtol=rtol, atol=atol, method=integration_method, t_eval=t_eval, pass_dy_as_arg=True)
    print(pysolve_sol.t.shape)
    chi_sci_sol = np.nansum((scipy_sol.y - correct_answer(scipy_sol.t, c1, c2))**2 / correct_answer(scipy_sol.t, c1, c2))
    chi_crykode_sol = np.nansum((cyrk_sol[1] - correct_answer(cyrk_sol[0], c1, c2))**2 / correct_answer(cyrk_sol[0], c1, c2))
    chi_pysolve_sol = np.nansum((pysolve_sol.y - correct_answer(pysolve_sol.t, c1, c2))**2 / correct_answer(pysolve_sol.t, c1, c2))
    
    print(f"SciPy (sol)\t|\tPySolve (sol)\t|\tcyrk_ode (sol)")
    print(f"{chi_sci_sol:0.5e}\t|\t{chi_pysolve_sol:0.5e}\t|\t{chi_crykode_sol:0.5e}")
    
    chi_sci_teval = np.nansum((scipy_teval.y - correct_answer(scipy_teval.t, c1, c2))**2 / correct_answer(scipy_teval.t, c1, c2))
    chi_crykode_teval = np.nansum((cyrk_teval[1] - correct_answer(cyrk_teval[0], c1, c2))**2 / correct_answer(cyrk_teval[0], c1, c2))
    chi_pysolve_teval = np.nansum((pysolve_teval.y - correct_answer(pysolve_teval.t, c1, c2))**2 / correct_answer(pysolve_teval.t, c1, c2))
    dense_sol = pysolve_sol(t_eval)
    chi_pysolve_dense = np.nansum((dense_sol - correct_answer(t_eval, c1, c2))**2 / correct_answer(t_eval, c1, c2))
    print()
    print(f"SciPy (teval)\t|\tPySolve (teval)\t|\tcyrk_ode (teval)\t|\tPySolve (dense)")
    print(f"{chi_sci_teval:0.5e}\t|\t{chi_pysolve_teval:0.5e}\t|\t{chi_crykode_teval:0.5e}\t|\t{chi_pysolve_dense:0.5e}")
    
    
#     fig1, ax1 = plt.subplots()
#     ax1.plot(scipy_sol.t, scipy_sol.y[0], c='b')
#     ax1.plot(scipy_sol.t, scipy_sol.y[1], c='r')
#     ax1.set(title="SciPy (sol)")
    
#     fig12, ax12 = plt.subplots()
#     ax12.plot(scipy_teval.t, scipy_teval.y[0], c='b')
#     ax12.plot(scipy_teval.t, scipy_teval.y[1], c='r')
#     ax12.set(title="SciPy (t-eval)")
    
#     fig2, ax2 = plt.subplots()
#     ax2.plot(pysolve_sol.t, pysolve_sol.y[0], c='b')
#     ax2.plot(pysolve_sol.t, pysolve_sol.y[1], c='r')
#     ax2.set(title="PySolve (sol)")
    
#     fig22, ax22 = plt.subplots()
#     ax22.plot(pysolve_teval.t, pysolve_teval.y[0], c='b')
#     ax22.plot(pysolve_teval.t, pysolve_teval.y[1], c='r')
#     ax22.set(title="PySolve (t-eval)")
    
#     fig23, ax23 = plt.subplots()
#     ax23.plot(t_eval, dense_sol[0], c='b')
#     ax23.plot(t_eval, dense_sol[1], c='r')
#     ax23.set(title="PySolve (dense)")
    
#     fig4, ax4 = plt.subplots()
#     ax4.plot(scipy_sol.t, (scipy_sol.y[0] - cyrk_sol[1][0]) / scipy_sol.y[0], c='b')
#     ax4.plot(scipy_sol.t, (scipy_sol.y[1] - cyrk_sol[1][1]) / scipy_sol.y[1], c='r')
#     ax4.set(title="SciPy (sol) - cyrk_ode")
    
#     fig5, ax5 = plt.subplots()
#     ax5.plot(scipy_sol.t, (scipy_sol.y[0] - pysolve_sol.y[0]) / scipy_sol.y[0], c='b')
#     ax5.plot(scipy_sol.t, (scipy_sol.y[1] - pysolve_sol.y[1]) / scipy_sol.y[1], c='r')
#     ax5.scatter(scipy_sol.t, np.zeros_like(scipy_sol.t), c='g', s=1)
#     ax5.scatter(pysolve_sol.t, np.zeros_like(scipy_sol.t), c='purple', s=5)
#     ax5.set(title="SciPy (sol) - pysolve")
    
#     fig5, ax5 = plt.subplots()
#     ax5.plot(cyrk_sol[0], (cyrk_sol[1][0] - pysolve_sol.y[0]) / cyrk_sol[1][0], c='b')
#     ax5.plot(cyrk_sol[0], (cyrk_sol[1][1] - pysolve_sol.y[1]) / cyrk_sol[1][1], c='r')
#     ax5.scatter(scipy_sol.t, np.zeros_like(scipy_sol.t), c='g', s=1)
#     ax5.scatter(pysolve_sol.t, np.zeros_like(scipy_sol.t), c='purple', s=5)
#     ax5.set(title="cyrk_ode - pysolve_sol")
    
#     sci_step_size = np.diff(scipy_sol.t)
#     solpy_step_size = np.diff(pysolve_sol.t)
#     cyrk_step_size = np.diff(cyrk_sol[0])
    
#     fig5, ax5 = plt.subplots()
# #     ax5.plot(scipy_sol.t, (scipy_sol.y[0] - pysolve_sol.y[0]) / scipy_sol.y[0], c='b', ls=':')
# #     ax5.plot(scipy_sol.t, (scipy_sol.y[1] - pysolve_sol.y[1]) / scipy_sol.y[1], c='r', ls=':')
#     ax5b = ax5.twinx()
#     ax5b.plot(scipy_sol.t[1:], sci_step_size, c='b', marker='.')
#     ax5b.plot(pysolve_sol.t[1:], solpy_step_size, c='r', marker='.')
# #     ax5b.plot(cyrk_sol[0][1:], cyrk_step_size, c='orange', marker='.')
#     ax5b.set(ylim=(0.3, 0.4))
#     plt.show()
    
#     break
    
    
    
    
    


 RTOL =  1e-08
(117,)
SciPy (sol)	|	PySolve (sol)	|	cyrk_ode (sol)
9.70354e-15	|	9.70355e-15	|	9.70346e-15

SciPy (teval)	|	PySolve (teval)	|	cyrk_ode (teval)	|	PySolve (dense)
-7.89887e-15	|	-7.89887e-15	|	3.81221e-06	|	-7.89887e-15

 RTOL =  1e-08
(117,)
SciPy (sol)	|	PySolve (sol)	|	cyrk_ode (sol)
9.70354e-15	|	9.70355e-15	|	9.70346e-15

SciPy (teval)	|	PySolve (teval)	|	cyrk_ode (teval)	|	PySolve (dense)
-7.89887e-15	|	-7.89887e-15	|	3.81221e-06	|	-7.89887e-15

 RTOL =  1e-08
(117,)
SciPy (sol)	|	PySolve (sol)	|	cyrk_ode (sol)
9.70354e-15	|	9.70355e-15	|	9.70346e-15

SciPy (teval)	|	PySolve (teval)	|	cyrk_ode (teval)	|	PySolve (dense)
-7.89887e-15	|	-7.89887e-15	|	3.81221e-06	|	-7.89887e-15

 RTOL =  1e-08
(117,)
SciPy (sol)	|	PySolve (sol)	|	cyrk_ode (sol)
9.70354e-15	|	9.70355e-15	|	9.70346e-15

SciPy (teval)	|	PySolve (teval)	|	cyrk_ode (teval)	|	PySolve (dense)
-7.89887e-15	|	-7.89887e-15	|	3.81221e-06	|	-7.89887e-15


C:\Users\joepr\AppData\Local\Temp\ipykernel_26928\2975167494.py:24: RuntimeWarning: invalid value encountered in divide
  chi_sci_sol = np.nansum((scipy_sol.y - correct_answer(scipy_sol.t, c1, c2))**2 / correct_answer(scipy_sol.t, c1, c2))
C:\Users\joepr\AppData\Local\Temp\ipykernel_26928\2975167494.py:25: RuntimeWarning: invalid value encountered in divide
  chi_crykode_sol = np.nansum((cyrk_sol[1] - correct_answer(cyrk_sol[0], c1, c2))**2 / correct_answer(cyrk_sol[0], c1, c2))
C:\Users\joepr\AppData\Local\Temp\ipykernel_26928\2975167494.py:26: RuntimeWarning: invalid value encountered in divide
  chi_pysolve_sol = np.nansum((pysolve_sol.y - correct_answer(pysolve_sol.t, c1, c2))**2 / correct_answer(pysolve_sol.t, c1, c2))
C:\Users\joepr\AppData\Local\Temp\ipykernel_26928\2975167494.py:31: RuntimeWarning: invalid value encountered in divide
  chi_sci_teval = np.nansum((scipy_teval.y - correct_answer(scipy_teval.t, c1, c2))**2 / correct_answer(scipy_teval.t, c1, c2))
C:\Users\joepr\App

In [12]:
print(pysolve_sol.y.shape)
print(pysolve_sol.t.shape)

(2, 147)
(147,)


In [8]:
pysolve_teval

In [4]:
print(pysolve_teval.y.shape)
print(pysolve_teval.t.shape)

(2, 100)
(100,)


In [5]:
pysolve_teval.t

array([ 0.        ,  0.1010101 ,  0.2020202 ,  0.3030303 ,  0.4040404 ,
        0.50505051,  0.60606061,  0.70707071,  0.80808081,  0.90909091,
        1.01010101,  1.11111111,  1.21212121,  1.31313131,  1.41414141,
        1.51515152,  1.61616162,  1.71717172,  1.81818182,  1.91919192,
        2.02020202,  2.12121212,  2.22222222,  2.32323232,  2.42424242,
        2.52525253,  2.62626263,  2.72727273,  2.82828283,  2.92929293,
        3.03030303,  3.13131313,  3.23232323,  3.33333333,  3.43434343,
        3.53535354,  3.63636364,  3.73737374,  3.83838384,  3.93939394,
        4.04040404,  4.14141414,  4.24242424,  4.34343434,  4.44444444,
        4.54545455,  4.64646465,  4.74747475,  4.84848485,  4.94949495,
        5.05050505,  5.15151515,  5.25252525,  5.35353535,  5.45454545,
        5.55555556,  5.65656566,  5.75757576,  5.85858586,  5.95959596,
        6.06060606,  6.16161616,  6.26262626,  6.36363636,  6.46464646,
        6.56565657,  6.66666667,  6.76767677,  6.86868687,  6.96

In [ ]:
A_my = np.asarray(((0.0), (0.0), (0.0), (0.0), (0.0), (0.2), (0.0), (0.0), (0.0), (0.0), (0.075), (0.225), (0.0), (0.0), (0.0), (0.9777777777777777), (-3.7333333333333334), (3.5555555555555554), (0.0), (0.0), (2.9525986892242035), (-11.595793324188385), (9.822892851699436), (-0.2908093278463649), (0.0), (2.8462752525252526), (-10.757575757575758), (8.906422717743473), (0.2784090909090909), (-0.2735313036020583)), dtype=np.float64)
A = np.array([
        [0, 0, 0, 0, 0],
        [1/5, 0, 0, 0, 0],
        [3/40, 9/40, 0, 0, 0],
        [44/45, -56/15, 32/9, 0, 0],
        [19372/6561, -25360/2187, 64448/6561, -212/729, 0],
        [9017/3168, -355/33, 46732/5247, 49/176, -5103/18656]
    ], dtype=np.longdouble)

In [11]:
A_my

array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.2       ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.075     ,   0.225     ,
         0.        ,   0.        ,   0.        ,   0.97777778,
        -3.73333333,   3.55555556,   0.        ,   0.        ,
         2.95259869, -11.59579332,   9.82289285,  -0.29080933,
         0.        ,   2.84627525, -10.75757576,   8.90642272,
         0.27840909,  -0.2735313 ])

In [23]:
np.set_printoptions(precision=32)
for a in A.flatten():
    print(f"    {a:0.32e},")

    0.00000000000000000000000000000000e+00,
    0.00000000000000000000000000000000e+00,
    0.00000000000000000000000000000000e+00,
    0.00000000000000000000000000000000e+00,
    0.00000000000000000000000000000000e+00,
    2.00000000000000011102230246251565e-01,
    0.00000000000000000000000000000000e+00,
    0.00000000000000000000000000000000e+00,
    0.00000000000000000000000000000000e+00,
    0.00000000000000000000000000000000e+00,
    7.49999999999999972244424384371086e-02,
    2.25000000000000005551115123125783e-01,
    0.00000000000000000000000000000000e+00,
    0.00000000000000000000000000000000e+00,
    0.00000000000000000000000000000000e+00,
    9.77777777777777745704668177495478e-01,
    -3.73333333333333339254522798000835e+00,
    3.55555555555555535818257339997217e+00,
    0.00000000000000000000000000000000e+00,
    0.00000000000000000000000000000000e+00,
    2.95259868922420354309110734902788e+00,
    -1.15957933241883850428166624624282e+01,
    9.822892851699435823320527